In [ ]:
import os
import socket
import gymnasium as gym
import json
import time
import subprocess
import struct, sys, time
import uuid
import threading
import gzip

from stable_baselines3 import PPO

from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
HOST = "127.0.0.1"
PORT = 27599
PROTOCOL_VERSION = 24

In [ ]:
def write_utf(s):
    encoded = s.encode("utf-8")
    length = len(encoded)
    return length.to_bytes(2, byteorder="big") + encoded

In [ ]:
packet_dict = {
    "PacketPing": 1,
    "PacketInitiateLogin": 2,  # Used this
    "PacketLogin": 3,          # Used this
    "PacketServerInfo": 4,
    "PacketHandshake": 5,
    "PacketKick": 6,
    "PacketQuit": 7,
    "PacketKeepConnected": 8,    # Might need this
    "PacketMessage": 9,
    "PacketPropertyColors": 10,
    "PacketCardCollectionData": 11,
    "PacketCardData": 12,
    "PacketCardActionRentData": 13,    # ACTION
    "PacketCardDescription": 14,       # ACTION
    "PacketCardPropertyData": 15,     
    "PacketCardBuildingData": 16,
    "PacketDestroyCardCollection": 17,
    "PacketDestroyCard": 18,
    "PacketPropertySetColor": 19,
    "PacketStatus": 20,
    "PacketMoveCard": 21,              # ACTION
    "PacketMovePropertySet": 22,       # ACTION?
    "PacketMoveRevealCard": 23,        # ACTION?
    "PacketMoveUnknownCard": 24,
    "PacketPlayerInfo": 25,
    "PacketPropertySetData": 26,
    "PacketUpdatePlayer": 27,
    "PacketDestroyPlayer": 28,
    "PacketRefresh": 29,
    "PacketUnknownCardCollectionData": 30,
    "PacketUndoCardStatus": 31,
    "PacketSoundData": 32,
    "PacketPlaySound": 33,
    "PacketPlayerButton": 34,
    "PacketDestroyButton": 35,
    "PacketInfoPlate": 36,
    "PacketDestroyInfoPlate": 37,
    "PacketCardButtons": 38,
    "PacketTurnOrder": 39,
    "PacketGameRules": 40,
    "PacketSetChatOpen": 41,
    "PacketRemoveMessageCategory": 42,
    "PacketSelectCardCombo": 43,           # ACTION
    "PacketSetAwaitingResponse": 44,
    "PacketChat": 45,
    "PacketActionAccept": 46,
    "PacketActionDraw": 47,                 # ACTION
    "PacketActionEndTurn": 48,              # ACTION
    "PacketActionMoveProperty": 49,         # ACTION
    "PacketActionChangeSetColor": 50,       # ACTION
    "PacketActionPay": 51,                  # ACTION
    "PacketActionPlayCardBuilding": 52,     # ACTION
    "PacketActionDiscard": 53,              # ACTION
    "PacketActionSelectPlayer": 54,         
    "PacketActionSelectProperties": 55,       
    "PacketActionSelectPlayerMonopoly": 56,
    "PacketActionUndoCard": 57,
    "PacketActionClickLink": 58,
    "PacketActionButtonClick": 59,
    "PacketActionUseCardButton": 60,
    "PacketActionRemoveBuilding": 61,           # ACTION
    "PacketActionSelectCardCombo": 62,          # ACTION
    "PacketActionMoveHandCard": 63,             # ACTION
    "PacketSoundCache": 64,
    "PacketActionStatePlayerTurn": 65,
    "PacketActionStateBasic": 66,
    "PacketActionStateRent": 67,
    "PacketActionStatePropertiesSelected": 68,
    "PacketActionStatePropertySetTargeted": 69,
    "PacketUpdateActionStateTarget": 70
}

## Initiate Login

In [ ]:
# PacketInitiateLogin ID = 2 (registered order in NetHandler), protocolVersion = 24
PACKET_ID = 2

sock = socket.create_connection((HOST, PORT))

# build payload: single int (protocolVersion)
payload = struct.pack(">i", PROTOCOL_VERSION)
payload_len = len(payload)

# Header as separate fields, not combined struct
header = struct.pack(">h", PACKET_ID) + struct.pack(">i", payload_len)
msg = header + payload

sock.sendall(msg)
print("Sent PacketInitiateLogin(protocolVersion=24)")

# try to read a response header (2 bytes id + 4 bytes len)
hdr = sock.recv(6)
if len(hdr) < 6:
    print("No response header received (got bytes):", hdr)
else:
    pkt_id, pkt_len = struct.unpack(">hI", hdr)
    print("Received packet id:", pkt_id, "payload length:", pkt_len)
    if pkt_len:
        data = b''
        while len(data) < pkt_len:
            chunk = sock.recv(pkt_len - len(data))
            if not chunk:
                break
            data += chunk
        print("Payload bytes:", data)

# keep socket open

## Login

In [ ]:
# PacketLogin ID = 3
PACKET_ID = 3
CLIENT_VERSION = "1.0.0"
PLAYER_NAME = "RLAgent"
UUID_BYTES = uuid.uuid4().bytes

# 1. protocolVersion (int)
proto_bytes = struct.pack(">i", PROTOCOL_VERSION)

# 2. clientVersion (String) - FIXED: int length prefix + UTF-16BE chars
client_version_chars = CLIENT_VERSION.encode('utf-16be')
client_version_payload = struct.pack(">i", len(CLIENT_VERSION)) + client_version_chars  # int length, not short

# 3. id (byte[] - 16 bytes)
id_payload = struct.pack(">i", 16) + UUID_BYTES  # Length prefix (16) + 16 bytes of UUID

# 4. name (String) - FIXED: int length prefix + UTF-16BE chars  
name_chars = PLAYER_NAME.encode('utf-16be')
name_payload = struct.pack(">i", len(PLAYER_NAME)) + name_chars  # int length, not short

# Combine all fields
payload = proto_bytes + client_version_payload + id_payload + name_payload

# Header: packet ID (short) + payload length (int)
header = struct.pack(">h", PACKET_ID) + struct.pack(">i", len(payload))
msg = header + payload

print(f"Payload length: {len(payload)}")
print(f"Client version payload: {client_version_payload.hex()}")
print(f"Name payload: {name_payload.hex()}")

In [ ]:
# Send the login packet
sock.sendall(msg)
print("Sent PacketLogin with corrected string format!")

# Read response
hdr = sock.recv(6)
if len(hdr) == 6:
    pkt_id = struct.unpack(">h", hdr[:2])[0]
    pkt_len = struct.unpack(">i", hdr[2:6])[0]
    print("Received packet id:", pkt_id, "payload length:", pkt_len)
    
    if pkt_len > 0:
        data = b''
        while len(data) < pkt_len:
            chunk = sock.recv(pkt_len - len(data))
            if not chunk:
                break
            data += chunk
        print("Login response payload:", data.hex())
    else:
        print("Login successful (no payload)")

## Keep Connected

In [ ]:
def send_keep_alive(sock):
    """Send keep-alive packet to prevent timeout"""
    # PacketKeepConnected is likely ID 7 (check your NetHandler order)
    KEEP_CONNECTED_ID = 8
    
    header = struct.pack(">h", 8) + struct.pack(">i", 0)  # No payload
    sock.sendall(header)
    # print("Sent keep-alive packet")

def start_keep_alive_loop(sock, interval=20):
    """Send keep-alive packets regularly"""
    while True:
        time.sleep(interval)
        try:
            send_keep_alive(sock)
        except:
            break  # Stop if connection is lost

In [ ]:
# Start keep-alive thread after successful login
keep_alive_thread = threading.Thread(
    target=start_keep_alive_loop, 
    args=(sock,), 
    daemon=True
)
keep_alive_thread.start()
print("Eternally connected")

In [ ]:
def listen_for_packets(sock):
    """Packet listener with decompression"""
    print("Starting packet listener...")
    while True:
        try:
            hdr = sock.recv(6)
            if len(hdr) < 6:
                print("Connection closed")
                break
                
            pkt_id = struct.unpack(">h", hdr[:2])[0]
            pkt_len = struct.unpack(">i", hdr[2:6])[0]
            
            print(f"Received packet ID: {pkt_id}, Length: {pkt_len}")
            
            if pkt_len > 0:
                data = b''
                while len(data) < pkt_len:
                    chunk = sock.recv(pkt_len - len(data))
                    if not chunk:
                        break
                    data += chunk
                
                # Handle PacketMessage (ID 9) - compressed messages
                if pkt_id == 9:
                    try:
                        # Skip first 4 bytes (they seem to be a header)
                        compressed_data = data[4:]
                        
                        # Decompress the gzip data
                        decompressed = gzip.decompress(compressed_data)
                        message = decompressed.decode('utf-8')
                        print(f"Server message: {message}")
                    except Exception as e:
                        print(f"Decompression failed: {e}")
                        print(f"Raw data: {data.hex()}")
                else:
                    print(f"Payload: {data.hex()}")
                    
        except Exception as e:
            print(f"Listener error: {e}")
            break

In [ ]:
# listener_thread = threading.Thread(
#     target=listen_for_packets, 
#     args=(sock,), 
#     daemon=True
# )
# listener_thread.start()

# print("Packet listener started. Now try your commands again...")

## Send Chat function

In [ ]:
def send_chat_command(sock, command):
    """Send a server command via chat (requires OP permissions first)"""
    message_bytes = command.encode('utf-16be')
    payload = struct.pack(">i", len(command)) + message_bytes
    header = struct.pack(">h", packet_dict["PacketChat"]) + struct.pack(">i", len(payload)) 
    sock.sendall(header + payload)

In [ ]:
# First make yourself OP, then add bot
# Give yourself OP permissions
send_chat_command(sock, f"/op {PLAYER_NAME}")

In [ ]:
# Add a bot named EasyBot
send_chat_command(sock, "/addbot Bot1")

In [ ]:
send_chat_command(sock, "/kick Logan")

In [ ]:
# Send command to start the game
send_chat_command(sock, "/start")

In [ ]:
# Have your bot check the turn order
send_chat_command(sock, "/listplayers")

In [ ]:
send_chat_command(sock, "/nextturn")

# Actions

In [ ]:
# # These are the REAL action packets your RL agent should use:
# def send_end_turn(sock):
#     """PacketActionEndTurn - ID 5"""
#     header = struct.pack(">h", 5) + struct.pack(">i", 0)
#     sock.sendall(header)

# def send_draw_card(sock):
#     """PacketActionDraw - ID 6"""  
#     header = struct.pack(">h", 6) + struct.pack(">i", 0)
#     sock.sendall(header)

# def send_play_card(sock, card_id):
#     """PacketActionPlayCard - ID 7 with card_id payload"""
#     payload = struct.pack(">i", card_id)  # Assuming card_id is int
#     header = struct.pack(">h", 7) + struct.pack(">i", len(payload))
#     sock.sendall(header + payload)

# def send_select_player(sock, player_id):
#     """PacketActionSelectPlayer - ID 8 with player_id payload"""
#     payload = struct.pack(">i", player_id)
#     header = struct.pack(">h", 8) + struct.pack(">i", len(payload))
#     sock.sendall(header + payload)

In [ ]:
# class MonopolyDealClient:
#     def __init__(self, host='localhost', port=27599):
#         self.host = host
#         self.port = port
#         self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#         self.sock.connect((host, port))
#         print("Connected to Monopoly Deal server!")

#     def send(self, message):
#         self.sock.sendall((message + "\n").encode())

#     def receive(self):
#         return self.sock.recv(4096).decode()

#     def close(self):
#         self.sock.close()


In [ ]:
# class MonopolyDealEnv(gym.Env):
#     def __init__(self, host='localhost', port=12345):  # Adjust port based on server
#         super().__init__()
#         # Start the server (optional, if not running in another terminal)
#         self.server_process = subprocess.Popen(['mvn', 'exec:java', '-pl', 'server'], cwd='server')
#         # Connect to the server
#         self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#         self.sock.connect((host, port))
#         # Define action and observation spaces (customize based on game)
#         self.action_space = gym.spaces.Discrete(10)  # Example: 10 possible actions (e.g., play card 0-9)
#         self.observation_space = gym.spaces.Dict({  # Example: game state as a dictionary
#             "board": gym.spaces.Box(low=0, high=100, shape=(10,), dtype=int),
#             "hand": gym.spaces.Box(low=0, high=100, shape=(5,), dtype=int)
#         })

#     def reset(self, seed=None, options=None):
#         # Send reset command (adjust based on server protocol)
#         self.sock.sendall("RESET\n".encode())  # Add \n if server expects it
#         response = self.sock.recv(1024).decode()
#         try:
#             state = json.loads(response)  # Assume JSON response
#         except json.JSONDecodeError:
#             state = response  # Fallback to raw text if not JSON
#         return state, {}

#     def step(self, action):
#         # Send action (adjust format based on server)
#         self.sock.sendall(f"PLAY_CARD {action}\n".encode())
#         response = self.sock.recv(1024).decode()
#         try:
#             data = json.loads(response)
#             state = data.get("state", {})
#             reward = data.get("reward", 0.0)
#             done = data.get("done", False)
#             info = data.get("info", {})
#         except json.JSONDecodeError:
#             # Fallback if response is not JSON
#             state = response
#             reward = 0.0
#             done = False
#             info = {}
#         return state, reward, done, False, info

#     def close(self):
#         self.sock.close()
#         if hasattr(self, 'server_process'):
#             self.server_process.terminate()

In [ ]:
# class MonopolyDealEnv(gym.Env):
#     def __init__(self, host='localhost', port=27599):
#         super().__init__()
#         self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#         self.sock.connect((host, port))
#         self.sock.sendall("addbot RLBot\n".encode())
#         self.action_space = gym.spaces.Discrete(20)
#         self.observation_space = gym.spaces.Dict({
#             "hand": gym.spaces.Box(low=0, high=100, shape=(5,), dtype=int),
#             "board": gym.spaces.Box(low=0, high=100, shape=(10,), dtype=int),
#             "turn": gym.spaces.Discrete(4)
#         })

#     def reset(self, seed=None, options=None):
#         self.sock.sendall("reset\n".encode())
#         time.sleep(0.1)
#         response = self.sock.recv(4096).decode()
#         try:
#             state = json.loads(response)
#         except json.JSONDecodeError:
#             state = {"raw": response}
#         self.sock.sendall("start\n".encode())
#         time.sleep(0.1)
#         response = self.sock.recv(4096).decode()
#         try:
#             state = json.loads(response)
#         except json.JSONDecodeError:
#             state = {"raw": response}
#         return state, {}

#     def step(self, action):
#         commands = {0: "nextturn", 1: "createcard action dealbreaker"}
#         command = commands.get(action, f"PLAY_CARD {action}")
#         self.sock.sendall(f"{command}\n".encode())
#         time.sleep(0.1)
#         response = self.sock.recv(4096).decode()
#         try:
#             data = json.loads(response)
#             state = data.get("state", {})
#             reward = data.get("reward", 0.0)
#             done = data.get("done", False)
#             info = data.get("info", {})
#         except json.JSONDecodeError:
#             state = {"raw": response}
#             reward = 0.0
#             done = False
#             info = {"error": "Non-JSON response"}
#         return state, reward, done, False, info

#     def close(self):
#         self.sock.sendall("stop\n".encode())
#         self.sock.close()




In [ ]:
# class MonopolyDealRLAgent:
#     def __init__(self, host='localhost', port=27599, username='RLAgent'):
#         self.sock = None
#         self.host = host
#         self.port = port
#         self.username = username
#         self.keep_alive_thread = None
        
#     def connect(self):
#         """Connect and login to server"""
#         self.sock = socket.create_connection((self.host, self.port))
        
#         # Send PacketInitiateLogin (ID 2)
#         self._send_packet(2, struct.pack(">i", 24))
        
#         # Send PacketLogin (ID 3)  
#         uuid_bytes = uuid.uuid4().bytes
#         client_version = "1.0.0"
        
#         payload = (
#             struct.pack(">i", 24) +  # protocolVersion
#             struct.pack(">i", len(client_version)) + client_version.encode('utf-16be') +  # clientVersion
#             struct.pack(">i", 16) + uuid_bytes +  # id
#             struct.pack(">i", len(self.username)) + self.username.encode('utf-16be')  # name
#         )
#         self._send_packet(3, payload)
        
#         # Start keep-alive to prevent timeout
#         self._start_keep_alive()
        
#     def _send_packet(self, packet_id, payload):
#         """Send packet with header"""
#         header = struct.pack(">h", packet_id) + struct.pack(">i", len(payload))
#         self.sock.sendall(header + payload)
        
#     def _start_keep_alive(self):
#         """Start keep-alive thread to prevent timeout"""
#         def keep_alive_loop():
#             while True:
#                 time.sleep(20)  # Send every 20 seconds
#                 try:
#                     self._send_packet(7, b'')  # PacketKeepConnected - ID 7
#                 except:
#                     break
                    
#         self.keep_alive_thread = threading.Thread(target=keep_alive_loop, daemon=True)
#         self.keep_alive_thread.start()
    
#     # RL Agent methods
#     def get_state(self):
#         """Parse incoming packets to get game state"""
#         # You'll need to implement packet parsing here
#         pass
        
#     def take_action(self, action_id, *args):
#         """Execute action in the game"""
#         if action_id == 0:  # End turn
#             self._send_packet(5, b'')  # PacketActionEndTurn
#         elif action_id == 1:  # Draw card
#             self._send_packet(6, b'')  # PacketActionDraw
#         # ... map other actions
        
#     def close(self):
#         """Clean up"""
#         if self.sock:
#             self.sock.close()

In [ ]:
# # Send a simple command
# sock.sendall("reset\n".encode())
# time.sleep(0.5)  # Wait for response
# response = sock.recv(4096).decode()
# print("Response:", response)

In [ ]:
# sock.sendall(b"addbot TestBot\n")  # Try adding a bot
# time.sleep(0.5)
# try:
#     response = sock.recv(1024)
#     print("Raw response (bytes):", response)
#     print("Decoded response:", response.decode('utf-8', errors='replace'))
# except socket.timeout:
#     print("No response received within 5 seconds")
# sock.close()

In [ ]:
# # Send a minimal message to mimic client handshake
# sock.sendall(b"HELLO\n")  # Placeholder, adjust if protocol is known
# time.sleep(0.5)

In [ ]:
# try:
#     response = sock.recv(1024)
#     print("Raw response (bytes):", response)
#     print("Decoded response:", response.decode('utf-8', errors='replace'))
#     # Try addbot after potential handshake
#     sock.sendall(b"addbot TestBot\n")
#     time.sleep(0.5)
#     response = sock.recv(1024)
#     print("Addbot response (bytes):", response)
#     print("Addbot decoded:", response.decode('utf-8', errors='replace'))
# except socket.timeout:
#     print("No response received within 5 seconds")
# sock.close()

In [ ]:
# sock.close()

In [ ]:
# env = MonopolyDealEnv()
# state, info = env.reset()
# print("Initial state:", state)
# next_state, reward, done, truncated, info = env.step(0)  # Example: nextturn
# print("Next state:", next_state, "Reward:", reward, "Done:", done, "Info:", info)
# env.close()

In [ ]:
# client = MonopolyDealClient()

# # Start the game
# # deals 5 cards to all players
# client.send("/start")
# print(client.receive())

In [ ]:
# client.send("/addbot Bot1")
# print(client.receive())

# client.send("/addbot Bot2")
# print(client.receive())

In [ ]:
# def start_game(self):
#     self.send("/start")
#     return self.receive()

# def add_bot(self, name):
#     self.send(f"/addbot {name}")
#     return self.receive()
